In [1]:
!pip3 install numpy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import sys
sys.path.append('./wrapper.cpython-313-x86_64-linux-gnu.so')

In [ ]:
import numpy as np
from wrapper import compute_min_sum_wrapper  # tu wrapper Cython

def show_matrix(L, pcm_matrix, rows, cols):
    print("Initial L Matrix:")
    for i in range(rows):
        row_str = []
        for j in range(cols):
            if pcm_matrix[i * cols + j] == 1:
                row_str.append(f"{L[i * cols + j]:.2f}")
            else:
                row_str.append("0.00")
        print(" ".join(row_str))


def main():
    with open("input3.txt", "r") as f:
        # Leer p
        p = float(f.readline().strip())
        p = 1.0 - (2.0/3.0) * p

        # Leer filas y columnas
        rows, cols = map(int, f.readline().split())

        # Inicializar matrices
        pcm_matrix = np.zeros((rows, cols), dtype=np.int32)
        L = np.zeros((rows, cols), dtype=np.float32)

        # Leer pcm_matrix y L
        for i in range(rows):
            values = list(map(int, f.readline().split()))
            for j, value in enumerate(values):
                pcm_matrix[i, j] = value
                if value == 1:
                    L[i, j] = p
                else:
                    L[i, j] = 0.0

        # Leer syndrome
        syndrome = np.zeros(cols,dtype=np.int32)
        values = list(map(int, f.readline().split()))
        for i,value in enumerate(values):
            syndrome[i] = value
        print("Initial Syndrome:", " ".join(map(str, syndrome)))

        # Inicializar Lj
        Lj = np.full(cols, p,np.float32)
        print(f"Lj: {p:.2f}")

        # Leer alpha
        alpha = float(f.readline().strip())
        print(f"Alpha: {alpha:.2f}")

        # Leer número de iteraciones
        num_it = int(f.readline().strip())
        print(f"Max Iterations: {num_it}\n")

        # Aplanar matrices
        L_flat = L.reshape(-1)
        #L_flat = L_flat.astype(np.float32)
        pcm_matrix_flat = pcm_matrix.reshape(-1)
        #pcm_matrix_flat = pcm_matrix_flat.astype(np.int32)

        # Mostrar L inicial (usando los vectores aplanados)
        show_matrix(L_flat, pcm_matrix_flat, rows, cols)

        # Llamar a la función de Cython con vectores aplanados
        compute_min_sum_wrapper(L_flat, pcm_matrix_flat, syndrome, rows, cols, Lj, alpha, num_it)


if __name__ == "__main__":
    main()

In [ ]:
import sys
sys.path.append('./wrapper.cpython-313-x86_64-linux-gnu.so')
import numpy as np
from wrapper import compute_min_sum_wrapper  # tu wrapper Cython

#TODO try to use an error model instead of randomly generating one
def random_error(pcm,rows,cols,per) -> np.ndarray:
    return np.random.binomial(1,per,size=cols)
    


def main():
    #read the B1_matrix and remove the commas
    pcm = []
    with open("B1_matrix.txt", "r") as f:
        rows = len(f.readlines())
        f.seek(0,0)
        for line in f:
            p = line.split(',')
            list_float = [float(i) for i in p]
            pcm.extend(list_float)
        
    cols = int(len(pcm)/441)
    p = 0.0
    p = (1.0 - (2.0/3.0) * p)
    alpha = 1.25
    print(rows)
    print(len(pcm))
    print(cols)
     
    NMCs = [10**4, 10**4, 10**4, 10**4, 10**4]  
    
    # Physical error rate that is simulated
    
    ps = np.linspace(0.001, 0.005, num=5)  

    for index, p_error in enumerate(ps):
        logical_errors = 0
        for iteration in range(NMCs[index]):
       
            error = random_error(pcm,rows,int(cols),p_error)
            #print(error)
            pcm = np.array(pcm, dtype=np.int32)
            syndrome = np.empty(rows, dtype=np.int32)

            for i in range(rows):
                row = pcm[i * cols:(i + 1)*cols].astype(np.int32) & 1
                syndrome[i] = (row @ error) & 1
            
            #print(syndrome)
            Lj = np.full(cols,p)
            num_it = 100
            L_flat = pcm.astype(np.float32).copy()
            L_flat = np.ascontiguousarray(L_flat)
            pcm = np.ascontiguousarray(pcm.astype(np.int32))
            syndrome = np.ascontiguousarray(syndrome.astype(np.int32))
            Lj = np.ascontiguousarray(Lj.astype(np.float32))
            error_computed = np.zeros(cols,dtype=np.int32)


            # Llamar a la función de Cython con vectores aplanados
            L_array = compute_min_sum_wrapper(L_flat, pcm.astype(np.int32), syndrome.astype(np.int32), rows, cols, Lj.astype(np.float32), alpha, num_it, error_computed)
            logical_error = (error + error_computed) %2
            if np.any(logical_error == 1):
                logical_errors += 1
        print("logical errors with physical error rate: ",p_error)
        print(logical_errors)
            



    

if __name__ == "__main__":
    main()

441
388962
882
logical errors with physical error rate:  0.001
0
logical errors with physical error rate:  0.002
0

USED ALL ITERATIONS WITHOUT FINDING THE ERRORlogical errors with physical error rate:  0.003
1
logical errors with physical error rate:  0.004
2

USED ALL ITERATIONS WITHOUT FINDING THE ERROR
USED ALL ITERATIONS WITHOUT FINDING THE ERROR
USED ALL ITERATIONS WITHOUT FINDING THE ERROR
USED ALL ITERATIONS WITHOUT FINDING THE ERRORlogical errors with physical error rate:  0.005
7
